## Notebook Goal:  
Using existing NLP and LDA methodologies to perform topic modeling on docket texts. Three hyperparameters to consider:
1. to remove organization or not in docket texts, so organizations themselves won't become topics.
2. to remove names or not in docket texts, so names themselves won't become topics.
3. variations in topic numbers: [2, 3, 5, 10]

Will then perform visualizations and model summary output on every permutation/iteration.

In [263]:
#standard libraries for basic operations
import pandas as pd
from bs4 import BeautifulSoup
import os
import codecs
import itertools as it
import warnings
import pickle
from collections import Counter

#NLP library
import spacy
nlp = spacy.load('en')

#LDA libraries
from gensim.models import Phrases #seems like this is slower, but Phaser was not compatible to our code? need some research
from gensim.models.word2vec import LineSentence
from spacy.lang.en.stop_words import STOP_WORDS
from gensim.corpora import DictSTOP_WORDSionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#visualization libraries
import pyLDAvis
import pyLDAvis.gensim


bookmark
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\spacy\data


c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:81: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:81: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:81: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:77: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\msgpack_numpy.py:81: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves sur

In [2]:
#import corpus/docket texts from html to pandas DataFrame
def grab_dockets():
    files = []
    #get all .html files in the folder (all docket files are in .html)
    for file in os.listdir('docket_texts/'):
        if file.endswith('.html'):
            files.append(os.path.join('docket_texts/', file))

    df_docket_texts = pd.DataFrame()
    
    for i in range(len(files)): #gather all docket texts
    #for i in [0, 1]: #for testing purposes
        
        content = codecs.open(files[i], 'r', 'utf-8').read()
        #use beautiful soup to get the case ID
        soup = BeautifulSoup(content, 'lxml')
        case_id = str(soup.find_all('h3'))    
        bookmark1 = case_id.find('CASE #:') + len('CASE #:')
        bookmark2 = case_id.find('</h3>')
        case_id = case_id[bookmark1:bookmark2]

        #use pandas to grab tables in the html files
        docket_tables = pd.read_html(content)

        #error checking: gotta do this because there's different length of docket_list/
        #usually docket texts are in docket_list[3], but not always
        n = 0
        while docket_tables[n].isin(['Docket Text']).sum().sum() == 0:
            #print(n, docket_tables[n].isin(['Docket Text']).sum().sum())
            n += 1
                        
        #print(i, files[i])
        #print(docket_tables[n].head())

        #docket_tables[n] is the docket text table
        new_header = docket_tables[n].iloc[0]
        docket_tables[n] = docket_tables[n][1:]
        docket_tables[n].columns = new_header
        
        docket_tables[n]['#'] = pd.to_numeric(docket_tables[n]['#'],
                                              downcast = 'signed', errors = 'coerce')
        docket_tables[n]['Date Filed'] = pd.to_datetime(docket_tables[n]['Date Filed'])
        docket_tables[n]['Case ID'] = case_id

        df_docket_texts = pd.concat([df_docket_texts, docket_tables[n]])
    #reorder a column
    cols = list(df_docket_texts.columns)
    df_docket_texts = df_docket_texts[[cols[-1]] + cols[:-1]]
    
    print('current docket text table size/shape: {}'.format(df_docket_texts.shape))
    return df_docket_texts

In [284]:
#current docket text table size/shape: (3244, 4), as of 2018-04-26
df = grab_dockets()
df.tail()

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprec

current docket text table size/shape: (3244, 4)


,Case ID,Date Filed,#,Docket Text
256,1:09-cv-08285-RA-FM,2013-09-24,216.0,STIPULATION OF DISMISSAL WITH PREJUDICE: that ...
257,1:09-cv-08285-RA-FM,2013-09-24,NaN,Terminate Transcript Deadlines (tn) (Entered: ...
258,1:09-cv-08285-RA-FM,2013-10-24,217.0,STIPULATION AND ORDER MODIFYING STIPULATED CON...
259,1:09-cv-08285-RA-FM,2015-05-14,218.0,NOTICE TO ATTORNEY TO RETRIEVE SEALED MATERIAL...
260,1:09-cv-08285-RA-FM,2015-05-18,219.0,SEALED MATERIALS RETRIEVED: Document(s) 95 wer...


In [285]:
docket_original = list(df['Docket Text'])
for i in range(5):
    print('docket text {}'.format(i))
    print(docket_original[i], '\n')

docket text 0
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment(s) added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016) 

docket text 1
Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. (Bowens, Priscilla) (Entered: 03/11/2016) 

docket text 2
Summons Issued as to Cardiogenics Holdings, Inc.. (Bowens, Priscilla) (Entered: 03/11/2016) 

docket text 3
NOTICE - emailed attorney regarding missing second page of the civil cover sheet. (Bowens, Priscilla) (Entered: 03/11/2016) 

docket text 4
In accordance with Rule 73 

In [9]:
docket_txt_filepath = 'docket_texts/docket_text_all.txt'

In [10]:
%%time

# writing our docket texts to a text file: docket_txt_filepath = 'docket_texts/docket_text_all.txt'
# if we need to update file, then True
if True:
    
    docket_count = 0

    # create & open a new file in write mode
    with codecs.open(docket_txt_filepath, 'w', encoding = 'utf_8') as docket_txt_file:


        for i in range(len(docket_original)):
            #note that I used replace here to get rid of '(s)' in the documents as SpaCy NLP wasn't able to handle it
            docket_txt_file.write(docket_original[i].replace('(s)', '') + '\n')
            docket_count += 1

    print('Text from {:,} docket texts written to the new txt file'.format(docket_count))
    
else:
    with codecs.open(docket_txt_filepath, encoding = 'utf_8') as docket_txt_file:
        for review_count, line in enumerate(docket_txt_file):
            pass
        
    print(u'Text from {:,} restaurant reviews in the txt file.'.format(docket_count))

Text from 3,244 docket texts written to the new txt file
Wall time: 13 ms


### SpaCy is doing a terrible job identifying Organization and Names. So we'll try Stanford NER

In [403]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
import os
java_path = 'C:/Program Files/Java/jdk-10.0.1/bin/java.exe'
os.environ['JAVAHOME'] = java_path

In [404]:
path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

output = []
for i in range(len(docket_original)):
    org_str = []
    name_str = []
    others_str = []
    tokens = nltk.tokenize.word_tokenize(docket_original[i])
    for label in tagger.tag(tokens):
        #print(label)
        if label[1] == 'ORGANIZATION':
            org_str.append(label[0])
            others_str.append('-ORG-')
        elif label[1] == 'PERSON':
            name_str.append(label[0])
            others_str.append('-NAME-')
        else:
            others_str.append(label[0])
    
    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(others_str)])

In [ ]:
df1 = pd.DataFrame(output, columns = ['Original Text', 'Org Part', 'Name Part', 'Stripped Txt'])
df1.to_csv('Org_Name_ID.csv')

### Can we effectively remove things with SpaCy NLP?

In [342]:
def text_preprocess1(text):
    text = text.replace('-', '')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('(s', 's')
    return text

def text_preprocess2(text):
    text = text.replace('.', '')
    return text

def lower_preprocess(word):
    if word.isupper():
        return word.lower()
    else:
        return word

keywords = pd.read_csv('docket_texts/keywords.csv', header = None)
keywords.columns = ['keywords']
keyword_list = list(keywords['keywords'])

['complaint against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 02078445206 Was the Disclosure Statement on Civil Cover Sheet completed yes,, filed by lg Capital Funding, llc. Steinmetz, Michael Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons Bowens, Priscilla . Entered: 03/10/2016',
 'Case assigned to Judge Ann m Donnelly and Magistrate Judge Vera m. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. Bowens, Priscilla Entered: 03/11/2016']

In [374]:
with codecs.open(docket_txt_filepath, encoding = 'utf_8') as f:
    entire_docket_text = text_preprocess1(f.read()).split('\n') #removing all the dashes to see what happens

print('# of docket texts available: {}'.format(len(entire_docket_text)))
print('\nSample docket texts:')
for i in range(5):
    print('{}: {}\n'.format(i + 1, entire_docket_text[i]))

entire_docket_text = [' '.join([lower_preprocess(word) for word in sentence.split()]) for sentence in entire_docket_text]

print('\nTransformed docket texts:')
for i in range(5):
    print('{}: {}\n'.format(i + 1, entire_docket_text[i]))

# of docket texts available: 3245

Sample docket texts:
1: COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 02078445206 Was the Disclosure Statement on Civil Cover Sheet completed YES,, filed by LG Capital Funding, LLC.  Steinmetz, Michael   Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons   Bowens, Priscilla .  Entered: 03/10/2016 

2: Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such.  Bowens, Priscilla   Entered: 03/11/2016 

3: Summons Issued as to Cardiogenics Holdings, Inc..  Bowens, Priscilla   Entered: 03/11/2016 

4: NOTICE  emailed attorney regarding missing second page of the civil cover sheet.  Bowens, Priscilla   Entered: 03/11/2016 

5: In accordance with Rule 73 of the Fe

### Q: If I lemmetize, are the orgs and names still there? A: Nope, they are gone. Gotta find another way to avoid lemmetizing till the very end

In [391]:
from difflib import SequenceMatcher

def rebuild(input_text, rule_dict, replace, exceptions):
    entire_output = []
    for sentence in input_text:
        output = []
        parsed_sentence = nlp(sentence)
        for token in parsed_sentence:
            #print('current token is: {}'.format(token.text))
            if token.text in rule_dict.keys():
                #print('in the rule_dict.keys():')
                if len(exceptions) == 0:
                    output.append(replace)
                    #print('exception rule is zero, so no exceptions, adding {}'.format(replace))
                else:
                    similar_score = max([SequenceMatcher(None, token.text.lower(), exception.lower()).ratio() for exception in exceptions])
                    #print('similar score: {}'.format(similar_score))
                    if similar_score > 0.7:
                        output.append(token.text)
                        #print('big enough so ill keep')
                    else:
                        output.append(replace)
                        #print('not enough, so replaced by {}'.format(replace))
            elif token.is_space or token.lemma_ in STOP_WORDS: #treatment for spaces and stopwords
                pass
            else:
                output.append(token.text)
                #print('regular add')
        entire_output.append(' '.join(output))
    return entire_output

In [392]:
#first level searching
org_list = []
name_list = []
date_list = []
punct_list = []
sym_list = []
num_list = []
space_list = []
pron_list = []
link_list = []

entire_docket_text = entire_docket_text[:10]

for sentence in entire_docket_text:
    parsed_sentence = nlp(sentence)
    for entity in parsed_sentence.ents:
        #print(entity.text, entity.label_)
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
        if entity.label_ == 'PERSON':
            name_list.append(entity.text)
        if entity.label_ == 'DATE':
            num_list.append(entity.text)
        if entity.label_ == 'CARDINAL':
            date_list.append(entity.text)
        
    for token in parsed_sentence:
        #print(token.text, token.text.find('http'))
        if token.pos_ == 'PUNCT':
            punct_list.append(token.text)
        if token.pos_ == 'SYM':
            sym_list.append(token.text)
        if token.pos_ == 'NUM':
            num_list.append(token.text)
        if token.is_space:
            space_list.append(token.text)
        if token.lemma_ == '-PRON-':
            pron_list.append(token.text)
        if token.text.find('http') == 0:
            link_list.append(token.text)

print('orgs: {}'.format(len(set(org_list))))
print('names: {}'.format(len(set(name_list))))
print('dates: {}'.format(len(set(date_list))))
print('puncts: {}'.format(len(set(punct_list))))
print('symbols: {}'.format(len(set(sym_list))))
print('#s: {}'.format(len(set(num_list))))
print('spaces: {}'.format(len(set(space_list))))
print('prons: {}'.format(len(set(pron_list))))
print('links: {}'.format(len(set(link_list))))

org_dict = Counter(org_list)
name_dict = Counter(name_list)
date_dict = Counter(date_list)
punct_dict = Counter(punct_list)
sym_dict = Counter(sym_list)
num_dict = Counter(num_list)
space_dict = Counter(space_list)
pron_dict = Counter(pron_list) #can safely remove pronouns, except for em
link_dict = Counter(link_list)

orgs: 10
names: 8
dates: 4
puncts: 5
symbols: 2
#s: 10
spaces: 0
prons: 4
links: 1


In [393]:
pron_dict

Counter({'You': 1, 'our': 1, 'their': 1, 'your': 1})

In [394]:
%%time
print('pre-processed: ')
for i, sentence in enumerate(entire_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

print('\n')
print('remove links: ')
output_docket_text = rebuild(entire_docket_text, link_dict, '-LINK-', []) #remove links
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))
    
print('\n')
print('remove pronouns: ')
output_docket_text = rebuild(output_docket_text, pron_dict, '-PRON-', ['em']) #remove pronouns
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

pre-processed: 
0: complaint against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 02078445206 Was the Disclosure Statement on Civil Cover Sheet completed yes,, filed by lg Capital Funding, llc. Steinmetz, Michael Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons Bowens, Priscilla . Entered: 03/10/2016
1: Case assigned to Judge Ann m Donnelly and Magistrate Judge Vera m. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. Bowens, Priscilla Entered: 03/11/2016
2: Summons Issued as to Cardiogenics Holdings, Inc.. Bowens, Priscilla Entered: 03/11/2016
3: notice emailed attorney regarding missing second page of the civil cover sheet. Bowens, Priscilla Entered: 03/11/2016
4: In accordance with Rule 73 of the Federal Rules of Civil Procedure and Local Rule 73.1, the parties 

In [395]:
%%time 

print('remove symbols: ')
output_docket_text = rebuild(output_docket_text, sym_dict, '', []) #remove symbols
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

remove symbols: 
0: complaint Cardiogenics Holdings , Inc. filing fee  400 , receipt number 02078445206 Disclosure Statement Civil Cover Sheet completed yes , , filed lg Capital Funding , llc . Steinmetz , Michael Additional attachment added 3/11/2016 :  1 Civil Cover Sheet ,  2 Proposed Summons Bowens , Priscilla . Entered : 03/10/2016
1: Case assigned Judge Ann m Donnelly Magistrate Judge Vera m. Scanlon . download review Individual Practices assigned Judges , located -PRON- website . Attorneys responsible providing courtesy copies judges -PRON- Individual Practices require . Bowens , Priscilla Entered : 03/11/2016
2: Summons Issued Cardiogenics Holdings , Inc .. Bowens , Priscilla Entered : 03/11/2016
3: notice emailed attorney regarding missing second page civil cover sheet . Bowens , Priscilla Entered : 03/11/2016
4: accordance Rule 73 Federal Rules Civil Procedure Local Rule 73.1 , parties notified parties consent United States magistrate judge court available conduct proceedings

In [396]:
%%time 
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

print('\n')
print('remove punctuation: ')
output_docket_text = [text_preprocess2(sentence) for sentence in output_docket_text]
output_docket_text = rebuild(output_docket_text, punct_dict2, '', keyword_list) #remove punctuation
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

0: complaint Cardiogenics Holdings , Inc. filing fee  400 , receipt number 02078445206 Disclosure Statement Civil Cover Sheet completed yes , , filed lg Capital Funding , llc . Steinmetz , Michael Additional attachment added 3/11/2016 :  1 Civil Cover Sheet ,  2 Proposed Summons Bowens , Priscilla . Entered : 03/10/2016
1: Case assigned Judge Ann m Donnelly Magistrate Judge Vera m. Scanlon . download review Individual Practices assigned Judges , located -PRON- website . Attorneys responsible providing courtesy copies judges -PRON- Individual Practices require . Bowens , Priscilla Entered : 03/11/2016
2: Summons Issued Cardiogenics Holdings , Inc .. Bowens , Priscilla Entered : 03/11/2016
3: notice emailed attorney regarding missing second page civil cover sheet . Bowens , Priscilla Entered : 03/11/2016
4: accordance Rule 73 Federal Rules Civil Procedure Local Rule 73.1 , parties notified parties consent United States magistrate judge court available conduct proceedings civil action inc

In [397]:
%%time
# second level searching, check results after transformation
org_list = []
name_list = []
date_list = []
punct_list = []
sym_list = []
num_list = []
space_list = []
pron_list = []
link_list = []

for sentence in output_docket_text:
    parsed_sentence = nlp(sentence)
    for entity in parsed_sentence.ents:
        #print(entity.text, entity.label_)
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
        if entity.label_ == 'PERSON':
            name_list.append(entity.text)
        if entity.label_ == 'DATE':
            num_list.append(entity.text)
        if entity.label_ == 'CARDINAL':
            date_list.append(entity.text)
        
    for token in parsed_sentence:
        if token.pos_ == 'PUNCT':
            punct_list.append(token.text)
        if token.pos_ == 'SYM':
            sym_list.append(token.text)
        if token.pos_ == 'NUM':
            num_list.append(token.text)
        if token.is_space:
            space_list.append(token.text)
        if token.lemma_ == '-PRON-':
            pron_list.append(token.text)
        if token.text.find('http') == 0:
            link_list.append(token.text)

print('orgs: {}'.format(len(set(org_list))))
print('names: {}'.format(len(set(name_list))))
print('dates: {}'.format(len(set(date_list))))
print('puncts: {}'.format(len(set(punct_list))))
print('symbols: {}'.format(len(set(sym_list))))
print('#s: {}'.format(len(set(num_list))))
print('spaces: {}'.format(len(set(space_list))))
print('prons: {}'.format(len(set(pron_list))))
print('links: {}'.format(len(set(link_list))))

org_dict = Counter(org_list)
name_dict = Counter(name_list)
date_dict = Counter(date_list)
punct_dict = Counter(punct_list)
sym_dict = Counter(sym_list)
num_dict = Counter(num_list)
space_dict = Counter(space_list)
pron_dict = Counter(pron_list) #can safely remove pronouns, except for em
link_dict = Counter(link_list)

orgs: 10
names: 9
dates: 4
puncts: 2
symbols: 0
#s: 10
spaces: 2
prons: 0
links: 0
Wall time: 123 ms


In [398]:
punct_dict

Counter({'-LINK-': 1, '-PRON-': 3})

In [399]:
%%time 
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))


output_docket_text = rebuild(output_docket_text, date_dict, '', keyword_list) #remove number and dates
output_docket_text = rebuild(output_docket_text, num_dict, '', keyword_list)

print('\n')
print('remove number and dates: ')
for i, sentence in enumerate(output_docket_text[:5]):
    print('{}: {}'.format(i, sentence))

0: complaint Cardiogenics Holdings  Inc filing fee 400  receipt number 02078445206 Disclosure Statement Civil Cover Sheet completed yes   filed lg Capital Funding  llc Steinmetz  Michael Additional attachment added 3/11/2016  1 Civil Cover Sheet  2 Proposed Summons Bowens  Priscilla Entered  03/10/2016
1: Case assigned Judge Ann m Donnelly Magistrate Judge Vera m Scanlon download review Individual Practices assigned Judges  located -PRON- website Attorneys responsible providing courtesy copies judges -PRON- Individual Practices require Bowens  Priscilla Entered  03/11/2016
2: Summons Issued Cardiogenics Holdings  Inc Bowens  Priscilla Entered  03/11/2016
3: notice emailed attorney regarding missing second page civil cover sheet Bowens  Priscilla Entered  03/11/2016
4: accordance Rule 73 Federal Rules Civil Procedure Local Rule 731  parties notified parties consent United States magistrate judge court available conduct proceedings civil action including jury nonjury trial order entry fi

In [400]:
%%time
# second level searching, check results after transformation
org_list = []
name_list = []
date_list = []
punct_list = []
sym_list = []
num_list = []
space_list = []
pron_list = []
link_list = []

for sentence in output_docket_text:
    parsed_sentence = nlp(sentence)
    for entity in parsed_sentence.ents:
        #print(entity.text, entity.label_)
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
        if entity.label_ == 'PERSON':
            name_list.append(entity.text)
        if entity.label_ == 'DATE':
            num_list.append(entity.text)
        if entity.label_ == 'CARDINAL':
            date_list.append(entity.text)
        
    for token in parsed_sentence:
        if token.pos_ == 'PUNCT':
            punct_list.append(token.text)
        if token.pos_ == 'SYM':
            sym_list.append(token.text)
        if token.pos_ == 'NUM':
            num_list.append(token.text)
        if token.is_space:
            space_list.append(token.text)
        if token.lemma_ == '-PRON-':
            pron_list.append(token.text)
        if token.text.find('http') == 0:
            link_list.append(token.text)

print('orgs: {}'.format(len(set(org_list))))
print('names: {}'.format(len(set(name_list))))
print('dates: {}'.format(len(set(date_list))))
print('puncts: {}'.format(len(set(punct_list))))
print('symbols: {}'.format(len(set(sym_list))))
print('#s: {}'.format(len(set(num_list))))
print('spaces: {}'.format(len(set(space_list))))
print('prons: {}'.format(len(set(pron_list))))
print('links: {}'.format(len(set(link_list))))

org_dict = Counter(org_list)
name_dict = Counter(name_list)
date_dict = Counter(date_list)
punct_dict = Counter(punct_list)
sym_dict = Counter(sym_list)
num_dict = Counter(num_list)
space_dict = Counter(space_list)
pron_dict = Counter(pron_list) #can safely remove pronouns, except for em
link_dict = Counter(link_list)

orgs: 11
names: 8
dates: 0
puncts: 2
symbols: 0
#s: 0
spaces: 1
prons: 0
links: 0
Wall time: 121 ms


In [402]:
print(org_dict)
print(name_dict)

Counter({'Complaint': 3, 'Individual Practices': 2, 'Kehrli': 1, 'Capital Funding': 1, 'Cardiogenics Holdings Inc': 1, 'Civil Cover Sheet  ': 1, ' Disclosure Statement Civil Cover Sheet': 1, 'Rule  Federal Rules': 1, 'Backend Note lg Capital': 1, 'Notice Conversion lg Capital': 1, 'Summons Issued Cardiogenics Holdings Inc': 1})
Counter({'Bowens Priscilla Entered': 6, 'Kevin Entered': 4, 'Steinmetz Michael': 1, 'Scanlon': 1, 'Vera': 1, 'Ann m': 1, 'Attached Notice': 1, 'Kevin Kehrli': 1})


In [157]:
[[nlp(sentence] for sentence in output_docket_text]

Counter({'December 6 , 2017': 1,
         '08/24/2010': 2,
         '2/16/2006': 2,
         'May 21 , 2001 to date': 1,
         '6/15/2006': 2,
         '8/12/02': 1,
         '69': 1,
         'March 20 , 2017': 2,
         '9/17/2004     ': 1,
         '10/17/2012': 2,
         '02/04/2011': 6,
         'theComplaint': 1,
         'August 9 , 2016': 1,
         '4/2/2012': 1,
         '2/7/2018': 1,
         'October 1 , 2010': 1,
         ' 04/22/2005': 1,
         '04/19/2012': 3,
         '5/5/2017': 2,
         'August 5,2011': 1,
         'March 18 , 2015': 1,
         'May 12 , 2015': 1,
         '9/15/2006': 2,
         'August 7 , 2013': 1,
         '7/7/2008': 1,
         'March 27': 4,
         '10/09/2007': 2,
         '10/23/2009': 2,
         'Exhibit 4258': 1,
         '3/27/2017': 3,
         '8/9/2006': 1,
         '11/05/2004': 1,
         'December 17 , 2013': 2,
         '2/4/2013     ago': 1,
         'December 6 , 2017     ': 1,
         '1052': 1,
         '9/

In [164]:
%%time
# second level searching, check results after transformation
org_list = []
name_list = []
date_list = []
punct_list = []
sym_list = []
num_list = []
space_list = []
pron_list = []

for sentence in output_docket_text:
    parsed_sentence = nlp(sentence)
    for entity in parsed_sentence.ents:
        #print(entity.text, entity.label_)
        if entity.label_ == 'ORG':
            org_list.append(entity.text)
        if entity.label_ == 'PERSON':
            name_list.append(entity.text)
        if entity.label_ == 'DATE':
            num_list.append(entity.text)
        if entity.label_ == 'CARDINAL':
            date_list.append(entity.text)
        
    for token in parsed_sentence:
        if token.pos_ == 'PUNCT':
            punct_list.append(token.text)
        if token.pos_ == 'SYM':
            sym_list.append(token.text)
        if token.pos_ == 'NUM':
            num_list.append(token.text)
        if token.is_space:
            space_list.append(token.text)
        if token.lemma_ == '-PRON-':
            pron_list.append(token.text)
        if token.text.find('http') == 0:
            link_list.append(token.text)

print('orgs: {}'.format(len(set(org_list))))
print('names: {}'.format(len(set(name_list))))
print('dates: {}'.format(len(set(date_list))))
print('puncts: {}'.format(len(set(punct_list))))
print('symbols: {}'.format(len(set(sym_list))))
print('#s: {}'.format(len(set(num_list))))
print('spaces: {}'.format(len(set(space_list))))
print('prons: {}'.format(len(set(pron_list))))
print('links: {}'.format(len(set(link_list))))

org_dict = Counter(org_list)
name_dict = Counter(name_list)
date_dict = Counter(date_list)
punct_dict = Counter(punct_list)
sym_dict = Counter(sym_list)
num_dict = Counter(num_list)
space_dict = Counter(space_list)
pron_dict = Counter(pron_list) #can safely remove pronouns, except for em
link_dict = Counter(link_list)

0.8235294117647058

In [158]:
pd.DataFrame([date_dict.keys(), date_dict.values()]).T.rename(columns = {0: 'IDed as dates', 1: 'count'}).to_csv('dates.csv', index = False)

In [ ]:
pd.DataFrame(set(org_list), columns = ['orgs'])

In [380]:
# extracting a review for testing
n_th_review = 0
with codecs.open(docket_txt_filepath, encoding = 'utf_8') as f:
    sample_review = list(it.islice(f, n_th_review, n_th_review + 1))[0]

print(sample_review)

#using NLP (SpaCy English) to parse the review text
parsed_review = nlp(sample_review)

#for example, we can print out the sentences from the parsed object
#why were the sentences split by a dash?!
for num, sentence in enumerate(parsed_review.sents):
    print('Sentence {}:'.format(num + 1))
    print(sentence)
    print('')

COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016)

Sentence 1:
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207

Sentence 2:
-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC.

Sentence 3:
(Steinmetz, Michael) (Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla).

Sentence 4:
(Entered: 03/10/2016)




In [381]:
#we can also look at entities, note that this result is not great
for num, entity in enumerate(parsed_review.ents):
    print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    print('')

Entity 1: COMPLAINT - ORG

Entity 2: Cardiogenics Holdings, Inc. - ORG

Entity 3: 400 - MONEY

Entity 4: the Disclosure Statement on Civil Cover Sheet - ORG

Entity 5: LG Capital Funding - ORG

Entity 6: LLC - PERSON

Entity 7: Steinmetz - PERSON

Entity 8: Michael - PERSON

Entity 9: 3/11/2016 - DATE

Entity 10: # 1 - MONEY

Entity 11: Civil Cover Sheet - ORG

Entity 12: Bowens - PERSON

Entity 13: Priscilla - PERSON

Entity 14: Entered - PERSON

Entity 15: 
 - GPE



In [382]:
#looking at parts of speech
token_text = [token.orth_ for token in parsed_review] #Verbatim text content 
token_lemma = [token.lemma_ for token in parsed_review] #lemmatization
token_pos = [token.pos_ for token in parsed_review] #part-of-speech.

pd.DataFrame(list(zip(token_text, token_lemma, token_pos)), columns=['token_text', 'lemma', 'part_of_speech'])
#remove when token.pos_ in ['PUNCT', 'SYM', 'NUM', 'PUNCT', 'SPACE']

,token_text,lemma,part_of_speech
0,COMPLAINT,complaint,PROPN
1,against,against,ADP
2,Cardiogenics,cardiogenics,PROPN
3,Holdings,holdings,PROPN
4,",",",",PUNCT
5,Inc.,inc.,PROPN
6,filing,filing,NOUN
7,fee,fee,NOUN
8,$,$,SYM
9,400,400,NUM


In [383]:
#try out lemmatization. this is quite important as it returns words into its original forms. However it's not perfect. We still see things such as 'attachment(s'
token_lemma = [token.lemma_ for token in parsed_review] # Base form of the token, with no inflectional suffixes.
token_shape = [token.shape_ for token in parsed_review] # Transform of the tokens's string, to show orthographic features. For example, "Xxxx" or "dd".

pd.DataFrame(list(zip(token_text, token_lemma, token_shape)), columns=['token_text', 'token_lemma', 'token_shape'])

,token_text,token_lemma,token_shape
0,COMPLAINT,complaint,XXXX
1,against,against,xxxx
2,Cardiogenics,cardiogenics,Xxxxx
3,Holdings,holdings,Xxxxx
4,",",",",","
5,Inc.,inc.,Xxx.
6,filing,filing,xxxx
7,fee,fee,xxx
8,$,$,$
9,400,400,ddd


In [385]:
# trying out different attributes
token_entity_type = [token.ent_type_ for token in parsed_review] #Named entity type.
token_entity_iob = [token.ent_iob_ for token in parsed_review] #IOB code of named entity tag. "B" means the token begins an entity, "I" means it is inside an entity, "O" means it is outside an entity, and "" means no entity tag is set.

pd.DataFrame(list(zip(token_text, token_entity_type, token_entity_iob)),
             columns=['token_text', 'entity_type', 'inside_outside_begin'])

,token_text,entity_type,inside_outside_begin
0,COMPLAINT,ORG,B
1,against,,O
2,Cardiogenics,ORG,B
3,Holdings,ORG,I
4,",",ORG,I
5,Inc.,ORG,I
6,filing,,O
7,fee,,O
8,$,,O
9,400,MONEY,B


In [386]:
# a bigger summary of the NLP tokens
token_attributes = [(token.orth_, #Verbatim text content
                     token.prob, #Smoothed log probability estimate of token's type
                     token.is_stop, #Is the token part of a "stop list"
                     token.is_punct, #Is the token punctuation
                     token.is_space, #Does the token consist of whitespace characters
                     token.like_num, #Does the token represent a number
                     token.is_oov) #Is the token out-of-vocabulary
                    for token in parsed_review]

df_temp = pd.DataFrame(token_attributes,
                       columns=['text', 'log_probability', 'stop?', 'punctuation?',
                                'whitespace?', 'number?', 'out of vocab.?'])

df_temp.loc[:, 'stop?':'out of vocab.?'] = (df_temp.loc[:, 'stop?':'out of vocab.?']
                                            .applymap(lambda x: u'Yes' if x else u''))
                                               
df_temp

,text,log_probability,stop?,punctuation?,whitespace?,number?,out of vocab.?
0,COMPLAINT,-20.0,,,,,Yes
1,against,-20.0,Yes,,,,Yes
2,Cardiogenics,-20.0,,,,,Yes
3,Holdings,-20.0,,,,,Yes
4,",",-20.0,,Yes,,,Yes
5,Inc.,-20.0,,,,,Yes
6,filing,-20.0,,,,,Yes
7,fee,-20.0,,,,,Yes
8,$,-20.0,,,,,Yes
9,400,-20.0,,,,Yes,Yes


In [408]:
# after exploring what SpaCy can do for us, we'll lemmatize our corpus using these functions
def unnecessary(token):
    """
    helper function to eliminate tokens
    when token.pos_ in ['PUNCT', 'SYM', 'NUM', 'PUNCT', 'SPACE'] or when the .lemma turns out to be ['-PRON-']
    """
    if token.pos_ in ['PUNCT', 'SYM', 'NUM', 'PUNCT', 'SPACE']:
        return True
    elif token.lemma_ in ['-PRON-']:
        return True
    else:
        return False

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename), batch_size = 10000, n_threads = 4):
        
        for sent in parsed_review.sents:
            #the problem is here where token.lemma_ transforms the pronouns
            yield ' '.join([token.lemma_ for token in sent if not unnecessary(token)])

### Step 0.c: Try gensim to do phrase modeling

1. Segment text of complete reviews into sentences & normalize text
2. First-order phrase modeling → apply first-order phrase model to transform sentences
3. Second-order phrase modeling → apply second-order phrase model to transform sentences
4. Apply text normalization and second-order phrase model to text of complete reviews

We'll use this transformed data as the input for some higher-level modeling approaches in the following sections.

In [411]:
unigram_sentences_filepath = 'docket_texts/unigram_sentences_all.txt'

In [412]:
%%time
# turn the lemmatized corpus into unigram sentences
with codecs.open(unigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for sentence in lemmatized_sentence_corpus(docket_txt_filepath):
        f.write(sentence + '\n')

Wall time: 47.9 s


In [413]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [414]:
#let's do some comparision between the original text and unigram sentences, shouldn't be that different.
print('original text:')
print(df['Docket Text'].iloc[0])
#print(df['Docket Text'].iloc[1])

print('\nunigram_sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
    print('')

original text:
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment(s) added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016)

unigram_sentence:
complaint against cardiogenics holdings inc. filing fee receipt number

be the disclosure statement on civil cover sheet complete -yes file by lg capital funding llc

steinmetz michael additional attachment add on civil cover sheet propose summon bowens priscilla

enter

case assign to judge ann m donnelly and magistrate judge vera m. scanlon

please download and review the individual practices of the assign judges locate on website

attorney be responsible for provide courtesy copy to judge where individual practices require such

bowens priscilla

enter

summon issue as to cardiogenics holdings inc



In [415]:
bigram_model_filepath = 'docket_texts/bigram_model_all' 

In [416]:
%%time

# store our bigram model
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk if we don't want to run this again
#bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 348 ms


In [417]:
bigram_sentences_filepath = 'docket_texts/bigram_sentences_all.txt'

In [418]:
%%time

# apply the bigram model, and write it to file
with codecs.open(bigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = ' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 796 ms


In [419]:
bigram_sentences = LineSentence(bigram_sentences_filepath)
print('unigram length = {}, bigram length = {}'.format(len(list(unigram_sentences)), len(list(bigram_sentences))))

unigram length = 15855, bigram length = 15855


In [420]:
#original v. unigram v. bigram. Some phrases should be combined already
start = 0
finish = 10
print('original text:')
print(df['Docket Text'].iloc[0])
print(df['Docket Text'].iloc[1])

print('\nunigram sentence:')
for unigram_sentence in it.islice(unigram_sentences, 0, 10):
    print(' '.join(unigram_sentence))
print('\nbigram sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))

original text:
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment(s) added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016)
Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. (Bowens, Priscilla) (Entered: 03/11/2016)

unigram sentence:
complaint against cardiogenics holdings inc. filing fee receipt number
be the disclosure statement on civil cover sheet complete -yes file by lg capital funding llc
steinmetz michael additional attachment add on civil cover sheet propose summon bowens priscilla
enter
case assign to judg

In [421]:
trigram_model_filepath = 'docket_texts/trigram_model_all'

In [422]:
%%time

# again, using Phrases to attach more words to phrases already formed
trigram_model = Phrases(bigram_sentences)
trigram_model.save(trigram_model_filepath)

# load the finished model from disk
#trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 314 ms


In [423]:
trigram_sentences_filepath = 'docket_texts/trigram_sentences_all.txt'

In [424]:
%%time

with codecs.open(trigram_sentences_filepath, 'w', encoding = 'utf_8') as f:
    for bigram_sentence in bigram_sentences:
        trigram_sentence = ' '.join(trigram_model[bigram_sentence])
        f.write(trigram_sentence + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 748 ms


In [425]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [426]:
start = 0
finish = 15
print('original text:')
print(df['Docket Text'].iloc[0],'\n')
print(df['Docket Text'].iloc[1],'\n')
print(df['Docket Text'].iloc[2],'\n')
print(df['Docket Text'].iloc[3],'\n')

print('\nUNIGRAM Sentence:')
for unigram_sentence in it.islice(unigram_sentences, start, finish):
    print(' '.join(unigram_sentence))
print('\nBIGRAM Sentence:')
for bigram_sentence in it.islice(bigram_sentences, start, finish):
    print(' '.join(bigram_sentence))
print('\nTRIGRAM Sentence:')
for trigram_sentence in it.islice(trigram_sentences, start, finish):
    print(' '.join(trigram_sentence))

original text:
COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment(s) added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016) 

Case assigned to Judge Ann M Donnelly and Magistrate Judge Vera M. Scanlon. Please download and review the Individual Practices of the assigned Judges, located on our website. Attorneys are responsible for providing courtesy copies to judges where their Individual Practices require such. (Bowens, Priscilla) (Entered: 03/11/2016) 

Summons Issued as to Cardiogenics Holdings, Inc.. (Bowens, Priscilla) (Entered: 03/11/2016) 

NOTICE - emailed attorney regarding missing second page of the civil cover sheet. (Bowens, Priscilla) (Entered: 03/11/2016) 


UNIGRAM Sentence:
complaint against cardiogenics holdings inc. filing fee receipt

In [427]:
#write trigram to file
trigram_dockets_filepath = 'docket_texts/trigram_transformed_dockets_all.txt'

### Step 1: Combine all NLP and phrase modeling preparation steps

1. Apply SpaCy NLP cleaning to the docket texts
2. Apply bigram model to the clean texts
3. Apply trigram model to the bigram outputs
4. Write to and export resulting file

In [428]:
%%time

#some feedbacks:
#remove all numbers and dates
#remove pronouns
#remove names
#lets throw the corresponding transformation back into parser again to see results

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
with codecs.open(trigram_dockets_filepath, 'w', encoding= 'utf_8') as f:

    for parsed_review in nlp.pipe(line_review(docket_txt_filepath), batch_size = 10000, n_threads = 4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review if not unnecessary(token)]

        # remove

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        print(bigram_review)
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review if term not in STOP_WORDS]

        #additionally we need to remove those pronouns
        #trigram_review = [term for term in trigram_review if term != '-PRON-']

        # write the transformed review as a line in the new file
        trigram_review = ' '.join(trigram_review)
        f.write(trigram_review + '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['complaint_against', 'cardiogenics_holdings', 'inc.', 'filing_fee', 'receipt_number', 'be', 'the', 'disclosure_statement', 'on', 'civil_cover', 'sheet_complete', '-yes', 'file', 'by', 'lg_capital', 'funding_llc', 'steinmetz', 'michael', 'additional', 'attachment_add', 'on', 'civil_cover', 'sheet', 'propose', 'summon', 'bowens_priscilla', 'enter']
['case', 'assign', 'to', 'judge', 'ann_m', 'donnelly', 'and', 'magistrate_judge', 'vera_m.', 'scanlon', 'please_download', 'and', 'review', 'the', 'individual_practices', 'of', 'the', 'assign_judges', 'locate', 'on', 'website', 'attorney', 'be', 'responsible_for', 'provide_courtesy', 'copy', 'to', 'judge', 'where_individual', 'practices_require', 'such', 'bowens_priscilla', 'enter']
['summon_issue', 'as', 'to', 'cardiogenics_holdings', 'inc', 'bowens_priscilla', 'enter']
['notice', 'email', 'attorney', 'regard', 'miss', 'second', 'page', 'of', 'the', 'civil_cover', 'sheet', 'bowens_priscilla', 'enter']
['in', 'accordance_with', 'rule', 'of', 

['scheduling_order', 'counsel', 'fail', 'to', 'appear', 'for', 'the', 'conference', 'schedule', 'for', 'today', 'on', 'or', 'before', 'counsel', 'be', 'to', 'submit_a', 'letter', 'explain', 'non', 'appearance', 'a', 'telephone_conference', 'be', 'set', 'for', 'at_am', 'before_magistrate', 'judge_vera', 'm._scanlon', "plaintiff_'s", 'counsel', 'be', 'to', 'organize', 'a', 'conference', 'call_then', 'phone', 'chamber', 'at', 'with', 'all', 'counsel', 'on', 'the', 'line', 'order', 'by', 'magistrate_judge', 'vera_m.', 'scanlon', 'on', 'gillespie', 'saudia', 'enter']
['letter', 'jointly', 'file', 'regard', 'the', 'missed', 'telephone_conference', 'by', 'lg_capital', 'funding_llc', 'kehrli_kevin', 'enter']
['motion', 'for', 'pre', 'motion', 'conference', 're', 'answer', 'to', 'complaint', 'counterclaim', 'by', 'lg_capital', 'funding_llc', 'attachment', 'exhibit', 'a', 'initial', 'pre', 'motion', 'letter', 'kehrli_kevin', 'enter']
['minute_entry', 'for', 'proceeding_hold', 'before_magistrate'

['letter', 'motion', 'for', 'leave', 'to', 'file', 'a', 'sur', 'reply', 'by', 'cardiogenics_holdings', 'inc', 'kogan_simon', 'enter']
['response', 'to', 'motion', 're', 'letter', 'motion', 'for', 'leave', 'to', 'file', 'a', 'sur', 'reply', 'file', 'by', 'lg_capital', 'funding_llc', 'kehrli_kevin', 'enter']
['order', 'deny', 'motion', 'for', 'leave', 'to', 'file', 'the', 'defendant', "'s", 'motion', 'for', 'leave', 'to', 'file', 'a', 'sur', 'reply', 'be', 'deny', 'order', 'by', 'judge', 'ann_m', 'donnelly', 'on', 'glass', 'caitlin', 'enter']
['minute_entry', 'for', 'proceeding_hold', 'before_magistrate', 'judge_vera', 'm._scanlon', 'telephone_conference', 'hold', 'on', 'in', 'room_north', 'kevin_kehrli', 'appear', 'for', 'plaintiff', 'simon_kogan', 'appear', 'for', 'defendant', 'see_separate', 'docket_entry', 'for', 'order', 'issue', 'from', 'this', 'proceeding', 'tape', 'quinlan_krista', 'enter']
['scheduling_order', 'as', 'discuss_during', 'the', 'conference', 'see_attached', 'order',

['order', 'deny', 'motion', 'for', 'sanctions', 'status_report', 'order', 'as', 'discuss_during', 'the', 'conference', 'see_attached', 'order', 'for', 'additional_detail', 'and', 'deadline', 'give', 'defendant', "'s", 'counsel', "'s", 'continue', 'nonappearance', 'and', 'no', 'participation', 'which', 'may', 'be', 'relate', 'to', 'serious', 'health', 'issue', 'defendant', 'must', 'retain', 'new', 'counsel', 'who', 'can', 'participate', 'the', 'action', 'be', 'stay', 'until', 'by', 'which', 'time', 'counsel', 'must', 'appear', 'for', 'the', 'corporation', 'if', 'counsel', 'fail', 'to', 'appear', 'plaintiff', 'may', 'move', 'for', 'a', 'certificate', 'of', 'default', 'and', 'default_judgment', 'the', 'address', 'and', 'contact', 'information', 'for', 'defendant', "'s", 'ceo', 'be', 'y.', 'gawad', 'ceo', 'cardiogenics_holdings', 'northern', 'drive', 'unit', 'mississauga', 'ontario', 'l4v1w8', 'canada', 'office', 'extension', 'cell', 'email', 'ygawad@cardiogenices.com', 'order', 'by', 'mag

...

['transcript', 'of', 'proceedings_re', 'conference_hold', 'on', 'before_magistrate', 'judge_frank', 'maas', 'court_reporter', 'transcriber', 'eve', 'giniger', 'transcript_may', 'be', 'view_at', 'the', 'court_public', 'terminal_or', 'purchase_through', 'the', 'court_reporter', 'transcriber_before', 'the', 'deadline', 'for', 'release', 'of', 'transcript_restriction', 'after_that', 'date', 'may', 'be', 'obtain_through', 'pacer', 'redaction_request', 'due', 'redact_transcript', 'deadline_set', 'for', 'release', 'of', 'transcript_restriction', 'set', 'for', 'kelly', 'enter']
['notice', 'of', 'filing', 'of', 'official_transcript', 'notice', 'be_hereby', 'give_that', 'an_official', 'transcript', 'of', 'a', 'conference', 'proceeding_hold', 'on', '9/9/11', 'have', 'be', 'file', 'by', 'the', 'court_reporter', 'transcriber', 'in', 'the', 'above_caption', 'matter', 'the', 'party', 'have_calendar', 'day', 'to', 'file', 'with', 'the', 'court', 'a', 'notice', 'of', 'intent', 'to', 'request_redaction'

['endorsed_letter', 'address', 'to', 'magistrate_judge', 'frank_maas', 'from_carletta', 'f.', 'higginson', 'date', 're', 'write', 'on_behalf', 'of', 'the', 'party', 'to', 'jointly', 'request', 'that', 'the', 'previously_schedule', 'may', 'conference', 'before', 'honor', 'be', 'reschedule', 'for', 'june', 'at_p.m.', 'as', 'inform', 'honor', "'s", 'law', 'clerk', 'yesterday', 'the', 'party', 'have', 'be', 'work', 'together', 'to', 'further', 'develop', 'the', 'issue', 'that', 'plan', 'to', 'submit', 'to', 'honor', 'and', 'believe', 'the', 'additional', 'time', 'request', 'will', 'provide', 'for', 'a', 'more', 'efficient', 'use', 'of', 'the', 'court', "'s", 'time', 'endorsement', 'grant', 'so_ordered', 'sign', 'by', 'magistrate_judge', 'frank_maas', 'on', 'djc', 'enter']
['set_reset', 'hearings', 'status_conference', 'set', 'for', 'at_pm', 'before_magistrate', 'judge_frank', 'maas', 'djc', 'enter']
['discovery', 'order', 'accordingly', 'brookfield', "'s", 'letter', 'request', 'that', 'aig

['notice', 'of', 'filing', 'of', 'official_transcript', 'notice', 'be_hereby', 'give_that', 'an_official', 'transcript', 'of', 'a', 'confernce', 'proceeding_hold', 'on', 'have', 'be', 'file', 'by', 'the', 'court_reporter', 'transcriber', 'in', 'the', 'above_caption', 'matter', 'the', 'party', 'have_calendar', 'day', 'to', 'file', 'with', 'the', 'court', 'a', 'notice', 'of', 'intent', 'to', 'request_redaction', 'of', 'this', 'transcript', 'if_no', 'such', 'notice', 'be', 'file', 'the', 'transcript_may', 'be', 'make_remotely', 'electronically_available', 'to', 'the', 'public_without', 'redaction_after', 'calendar_day', '(mcguirk_kelly', 'enter']
['order', 'as', 'may', 'be', 'aware', 'this', 'case', 'have', 'be', 'reassign', 'to', 'docket', 'all', 'current', 'deadline', 'schedule', 'and', 'order', 'remain', 'in', 'effect', 'include', 'referral', 'of', 'the', 'case', 'to', 'magistrate_judge', 'maas', 'for', 'general', 'pretrial', 'proceeding', 'and', 'the', 'july', 'second', 'revise_schedu

['fifth', 'revise_scheduling', 'order', 'the', 'party', 'to', 'this_action', 'hereby', 'submit', 'this', 'fifth', 'revise_scheduling', 'order', 'fifth', 'revise_scheduling', 'order', 'this', 'fifth', 'revise_scheduling', 'order', 'amend', 'the', 'relevant', 'provision', 'ofthe', 'civil', 'case_management', 'plan', 'and', 'scheduling_order', 'date', 'february', 'the', 'second', 'revise_scheduling', 'order', 'date', 'july', 'the', 'third', 'revise_scheduling', 'order', 'date', 'september', 'and', 'the', 'fourth', 'revise_scheduling', 'order', 'date', 'october', 'as', 'follow', 'plaintiff', "'", 'expert', 'disclosure', 'as', 'require', 'by', 'federal_rule', 'of', 'civil_procedure', 'shall', 'be', 'serve', 'by', 'december', "defendant_'", 'expert', 'disclosure', 'as', 'require', 'by', 'federal_rule', 'of', 'civil_procedure', 'shall', 'be', 'serve', 'by', 'april', 'will', 'plaintiff', "'", 'rebuttal', 'expert_report', 'if_any', 'shall', 'be', 'serve', 'by', 'may', 'all', 'expert_discovery',

['endorsed_letter', 'address', 'to', 'magistrate_judge', 'frank_maas', 'from', 'brian', 'j.', 'fischer', 'date', 're', 'represent', 'the', 'plaintiff', 'brookfield', 'in', 'the', 'above_caption', 'matter', 'write', 'pursuant', 'to', 'rule', 'of', 'honor', "'s", 'individual_practices', 'and', 'local_civil', 'rule', 'to', 'request', 'an', 'informal', 'conference', 'to', 'address', 'dispute', 'term', 'of', 'an', 'agreement', 'the', 'party', 'reach', 'earlier', 'this', 'year', 'that', 'be', 'intend', 'to', 'resolve', 'a', 'discovery_dispute', 'endorsement', 'the', 'court', 'will', 'hold', 'a', 'telephone_conference', 'to', 'discuss', 'this', 'issue', 'on', 'at_a.m.', "plaintiff_'s", 'counsel', 'should', 'initiate', 'the', 'call', 'telephone_conference', 'set', 'for', 'at_am', 'before_magistrate', 'judge_frank', 'maas', 'sign', 'by', 'magistrate_judge', 'frank_maas', 'on', 'djc', 'enter']
['endorsed_letter', 'address', 'to', 'magistrate_judge', 'frank_maas', 'from', 'brian', 'j.', 'fischer'

In [429]:
# extracting a review for comparision purposes
n_th_review = 0
with codecs.open(docket_txt_filepath, encoding = 'utf_8') as f:
    sample_review = list(it.islice(f, n_th_review, n_th_review + 1))[0]
    
print('Original:' + '\n')
print(sample_review)

    
# the transformed file shouldn't have any pronouns in it
print('----' + '\n')
print('Transformed:' + '\n') 

with codecs.open(trigram_dockets_filepath, encoding = 'utf_8') as f:
    for review in it.islice(f, n_th_review, n_th_review + 1):
        print(review)

Original:

COMPLAINT against Cardiogenics Holdings, Inc. filing fee $ 400, receipt number 0207-8445206 Was the Disclosure Statement on Civil Cover Sheet completed -YES,, filed by LG Capital Funding, LLC. (Steinmetz, Michael) (Additional attachment added on 3/11/2016: # 1 Civil Cover Sheet, # 2 Proposed Summons) (Bowens, Priscilla). (Entered: 03/10/2016)

----

Transformed:

complaint_against cardiogenics_holdings_inc. filing_fee_receipt_number disclosure_statement civil_cover_sheet_complete -yes file lg_capital_funding_llc steinmetz michael additional_attachment_add civil_cover_sheet propose summon bowens_priscilla enter



### Step 2.a: Loading trigram texts into a dictionary. Filter out some extreme values along the way

In [407]:
trigram_dictionary_filepath = 'docket_texts/trigram_dict_all.dict'

In [408]:
%%time

#some dictionary hyperparameters:
no_below = 10 #reference is 10
no_above = 0.4 #reference is 0.4



trigram_reviews = LineSentence(trigram_dockets_filepath)

# learn the dictionary by iterating over all of the reviews
trigram_dictionary = Dictionary(trigram_reviews)

# filter tokens that are very rare otrigram_reviewsr too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below = no_below, no_above = no_above) #this step is questionable. May need to change the parameters
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
#trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 129 ms


### Step 2.b: Load into bag-of-words

In [409]:
trigram_bow_filepath = 'docket_texts/trigram_bow_corpus_all.mm'

In [410]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [411]:
%%time

# generate bag-of-words representations for
# all reviews and save them as a matrix
MmCorpus.serialize(trigram_bow_filepath, trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 310 ms


In [412]:
lda_model_filepath = 'docket_texts/lda_model_all'

### Step 2.c LDA Model Training
note that we need couple of things:
1. dictionary
2. bow corpus
3. numbre of topics

### During LDA modeling, some parameters we may need to be mindful of:
- alpha (parameter of the Dirichlet pior of the per-document topic distribution)
   - high: each document will contain many topics
   - low: each document iwll have distinct topics

- beta (parameter of the Dirichlet prior on the per-topic word distribution)
   - high: each topic will contain many words
   - low: each topic will contain few words

- theta (topic distribution for document m)
- z (topic for the n-th word in document m)
- w (specific word)

In [413]:
%%time

#note that we can change # of topics any time
num_topics = 10

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    # workers => sets the parallelism, and should be
    # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus, num_topics = num_topics, id2word = trigram_dictionary, workers = 4)

lda.save(lda_model_filepath)

# load the finished LDA model from disk
#lda = LdaMulticore.load(lda_model_filepath)

Wall time: 23.2 s


### Step 2.d This is to check out which tokens are assigned to which topic. Use Intuition to provide a description for the topic

In [414]:
def explore_topic(topic_number, topn = 10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format('term', 'frequency') + '\n')

    for term, frequency in lda.show_topic(topic_number, topn = topn):
        print('{:20} {:.3f}'.format(term, round(frequency, 3)))

In [415]:
# try looking at different topics' constitutinos
# we might want to 
for i in range(5):
    print("\n topic {}'s tokens:".format(i))
    explore_topic(topic_number = i)
#seems like topic:
#0: 
#1: 
#2: 
#3: 
#4: 
#etc...


 topic 0's tokens:
term                 frequency

order                0.046
sign                 0.018
motion               0.015
party                0.013
answer_due           0.011
defendant            0.010
judgment             0.010
date                 0.009
shall                0.009
schedule             0.009

 topic 1's tokens:
term                 frequency

exhibit              0.222
motion               0.036
order                0.024
dismiss              0.016
notice               0.015
case                 0.011
date                 0.009
declaration          0.008
affidavit            0.008
grant                0.008

 topic 2's tokens:
term                 frequency

exhibit              0.216
motion               0.021
order                0.020
declaration          0.018
kehrli_kevin         0.011
attachment           0.011
memorandum           0.011
's                   0.010
support              0.010
b                    0.010

 topic 3's tokens:
term          

### Step 2.e After looking at the token makeup for each topic, one can interpret a description for each topic. This will aid in future visualizations

In [416]:
#so these are fake as I didn't go through all the outputs
#until we provided a label:
topic_names = {}
for i in range(num_topics):
    topic_names[i] = 'Topic ' + str(i)

''' this is a place holder
topic_names = {0: 'mexican',
               1: 'menu',
               2: 'thai',
               3: 'steak',
               4: 'donuts & appetizers',
               5: 'specials',
               6: 'soup',
               7: 'wings, sports bar',
               8: 'foreign language',
               9: 'las vegas'}
'''
print(topic_names)

{0: 'Topic 0', 1: 'Topic 1', 2: 'Topic 2', 3: 'Topic 3', 4: 'Topic 4', 5: 'Topic 5', 6: 'Topic 6', 7: 'Topic 7', 8: 'Topic 8', 9: 'Topic 9'}


In [417]:
topic_names_filepath = 'docket_texts/topic_names.pkl'

with open(topic_names_filepath, 'wb') as f:
    pickle.dump(topic_names, f)

### Step 3 Proceed to use LDavis to visualize our results

In [418]:
LDAvis_data_filepath = 'docket_texts/ldavis_prepared'

In [419]:
%%time

LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus, trigram_dictionary)

with open(LDAvis_data_filepath, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)

Wall time: 22.9 s


In [420]:
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

### What are we looking at?
- the plot is redered in 2D according to MDS algo. Topics that are similar should appear close together; those that are dis-similar should be far apart.
- the relative size of a topic's circle in the plot corresponds to the relative frequency of the topic in the corpus
- when no topic is selected, the bar chart changes to show the top 30 most relevant terms for the selected topic. the relevance metric is controlled by the parameter λ. 
- λ close to 1 will rank the terms soley according to their probability within the topic.
- λ close to 0 will rank the terms solely according to their distinctiveness/exclusivity within the topic. (terms that only occur in this topic)
- Rolling over a term in the bar chart will cause the topic to resize, thereby showing the strength of the relationship between the topic and the selected term.

In [421]:
pyLDAvis.display(LDAvis_prepared)

### Step 4 Let's label some texts with topics

In [422]:
def get_sample_docket(docket_number):
    return list(it.islice(line_review(docket_txt_filepath), docket_number, docket_number + 1))[0]

In [423]:
def lda_description(docket_text, min_topic_freq = 0.05):
    '''
    accept the original text of a review and 
    1) parse it with spaCy,
    2) apply text pre-proccessing steps, 
    3) create a bag-of-words representation, 
    4) create an LDA representation, and
    5) print a sorted list of the top topics in the LDA representation
    '''
    output = []
    # parse the review text with spaCy
    parsed_docket = nlp(docket_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_docket = [token.lemma_ for token in parsed_review if not unnecessary(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_docket = bigram_model[unigram_docket]
    trigram_docket = trigram_model[bigram_docket]
    
    # remove any remaining stopwords
    trigram_docket = [term for term in trigram_docket if term not in STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_docket)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda.sort(key = lambda tup: tup[1], reverse = True)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        #print('{:25} {}'.format(topic_names[topic_number], round(freq, 3)))
        output.append((topic_names[topic_number], round(freq, 5)))
    return output

In [424]:
sample_docket = get_sample_docket(2)
print(sample_docket)
print(lda_description(sample_docket))

Summons Issued as to Cardiogenics Holdings, Inc.. (Bowens, Priscilla) (Entered: 03/11/2016)



c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[('Topic 0', 0.1), ('Topic 1', 0.1), ('Topic 2', 0.1), ('Topic 3', 0.1), ('Topic 4', 0.1), ('Topic 5', 0.1), ('Topic 6', 0.1), ('Topic 7', 0.1), ('Topic 8', 0.1), ('Topic 9', 0.1)]


In [425]:
df['topics'] = df['Docket Text'].apply(lda_description)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


### Aside - Examining imported keywords

In [426]:
keywords = pd.read_csv('docket_texts/keywords.csv', header = None)
keywords.columns = ['keywords']
keyword_list = list(keywords['keywords'])
print(keyword_list)

['acknowledge', 'acknowledgement', 'adjoure', 'Adjourments', 'adjourments', 'affidavit', 'amended', 'amended pleading', 'answer', 'appeal', 'arbitration', "attorney's fee", "attorney's fees", "attorneys' fees", "attorneys' fees", 'bench', 'brief', 'certificate', 'certificate of default', 'certificate of service', 'citizen', 'citizenship', "clerk's default certificate", 'complaint', 'conference', 'counter', 'counterclaim', 'cross motion', 'cross-motion', 'damages', 'decision', 'decision and order', 'declaration', 'default', 'default judgment', 'direct', 'directing', 'discovery', 'discovery motion', 'discovery plan', 'diversity', 'elements', 'endorsed', 'entry', 'ex parte', 'exhibit', 'exhibits', 'extension', 'grant', 'in limine', 'injunction', 'injunctive relief', 'Interrogatories', 'interrogatories', 'joint', 'joint pretrial order', 'judgment', 'jurisdiction', 'jury', 'leave', 'Leave', 'Letter', 'letter motions', 'memorandum', 'moot', 'motion', 'motion to dismiss', 'non jury', 'non-jur

In [427]:
def count_keywords(some_string):
    count = 0
    matches = []
    for keyword in keyword_list:
        if keyword in some_string:
            count += 1
            matches.append(keyword)
    #print(matches)
    return count, matches

In [428]:
df['count keywords'] = df['Docket Text'].apply(count_keywords)

In [429]:
df.head()

,Case ID,Date Filed,#,Docket Text,topics,count keywords
1,1:16-cv-01215-AMD-SJB,2016-03-10,1.0,"COMPLAINT against Cardiogenics Holdings, Inc. ...","[(Topic 0, 0.1), (Topic 1, 0.1), (Topic 2, 0.1...","(0, [])"
2,1:16-cv-01215-AMD-SJB,2016-03-10,NaN,Case assigned to Judge Ann M Donnelly and Magi...,"[(Topic 0, 0.1), (Topic 1, 0.1), (Topic 2, 0.1...","(0, [])"
3,1:16-cv-01215-AMD-SJB,2016-03-10,2.0,"Summons Issued as to Cardiogenics Holdings, In...","[(Topic 0, 0.1), (Topic 1, 0.1), (Topic 2, 0.1...","(0, [])"
4,1:16-cv-01215-AMD-SJB,2016-03-11,NaN,NOTICE - emailed attorney regarding missing se...,"[(Topic 0, 0.1), (Topic 1, 0.1), (Topic 2, 0.1...","(0, [])"
5,1:16-cv-01215-AMD-SJB,2016-03-11,3.0,In accordance with Rule 73 of the Federal Rule...,"[(Topic 0, 0.1), (Topic 1, 0.1), (Topic 2, 0.1...","(5, [entry, judgment, jury, nonjury, trial])"


In [430]:
#export to csv
df.to_csv('docket_texts/docket_text_topics.csv')